# Atividade 2 - Evolução de Software

# Equipe:
- Adicley de Oliveira Costa
- Alíria de Santana Amorim Cruz
- Herbert Barreto Freire
- Iago Soares de Macedo
- Letícia Caroline da Silva Oliveira
- Lucas Aragão Damaceno
- Rodrigo Nunes de Santana

# Bibliotecas necessárias

In [ ]:
!pip install transformers

# Etapas para execução da atividade

- Importação das bibliotecas necessárias e listagem dos modelos escolhidos para a atividade

In [ ]:
import os
import json as js
from transformers import pipeline, AutoModel

models = [
    "microsoft/Phi-3-mini-128k-instruct",
    "microsoft/Phi-4-mini-instruct",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
]

- Diretórios necessários para leitura dos artefatos do código fonte e para salvar o vetor de respostas de cada modelo selecionado

In [ ]:
#Dirs;
artefacts_path = "source_code_artefacts"
model_resp_path = "models_code_smells_resp"

#Nome do arquivo de saida para as respostas dos modelos;
models_resp_file_name = "models_resp.json"

#Regata o nome dos modulos para analise dos arquivos
modules_name = os.listdir(artefacts_path)
modules_name = [module for module in modules_name if module != ".ipynb_checkpoints"]

- Funções necessárias para execução da atividade

In [ ]:
def read_file(artefact_dir):
    with open(artefact_dir, 'r') as f:
        return f.read()

def write_json_file(path, data):
    with open(path, 'w') as f:
        js.dump(data, f, indent=4)

def make_json_models_resp(models_resp):
    dir = f"{model_resp_path}/{models_resp_file_name}"
    if not os.path.exists(model_resp_path):
        os.mkdir(model_resp_path)
    write_json_file(dir, models_resp)

def model_txt_gen(pipe, code, artefact_name, code_smells, model_template):
    _prompt = prompt.format(str(code_smells), code)
    model_resp = pipe(_prompt)[0]['generated_text']
    model_resp_template = {'artefact_name': artefact_name, 'model_resp': model_resp}
    model_template['model_results'].append(model_resp_template)

- Lista com alguns code smells possíveis

In [ ]:
code_smells = [
    "Long Method", "Large Class", "Primitive Obsession", "Long Parameter List", "Data Clumps", "Switch Statements",
    "Divergent Change", "Parallel Inheritance Hierarchies", "Shotgun Surgery", "Duplicate Code", "Dead Code",
    "Speculative Generality", "Lazy Class", "Inappropriate Intimacy", "Incomplete Library Class", "Message Chains"
    "Feature Envy"
]

- Definição do prompt para identificar se os códigos possuem algum tipo de code smells listados

In [ ]:
prompt = """You are code analysis assistant.
            Identify if the source code has any of the following code smells: {}
            Code: {}
            Provide your response in the exact format: "Smell: <name>".
            Do not add explanations.
            Response:"""

- Vetor de resposta para cada modelo

In [ ]:
models_resp = []
task = "text-generation"
device = "cuda"
temperature = 0.5

for module_name in modules_name:
    module_template = {'module_name': module_name, 'models_analysis':[]}
    print(f"\n*** Module_name: {module_name}")
    artefacts_names = os.listdir(f"{artefacts_path}/{module_name}")
    for model in models:
        i = 0
        pipe = pipeline(task=task, model=model, device=device, temperature=temperature, return_full_text=False, do_sample=True)
        model_template = {'model_name': model, 'model_results': []}
        print(f"\n - Model: {model}")
        print(" - Artefacts:", end=' ')
        for artefact_name in artefacts_names:
            code = read_file(f"{artefacts_path}/{module_name}/{artefact_name}")
            model_txt_gen(pipe, code[:1024], artefact_name, code_smells, model_template)
            print(i, end=' ')
            i += 1
        module_template['models_analysis'].append(model_template)
    models_resp.append(module_template)

- Geração do arquivo de respostas contendo a identificação de code smells por cada modelos listado

In [ ]:
make_json_models_resp(models_resp)